In [ ]:
import matplotlib.pyplot as plt
import json
import numpy as np
import yaml
import os
from sklearn import datasets
import matplotlib.gridspec as gridspec
cwd = os.getcwd()

green = plt.cm.Greens(0.2*(6/4) + 0.4)
blue = plt.cm.Blues(0.2*(6/4) + 0.4)
red = plt.cm.Reds(0.2*(6/4) + 0.4)

In [ ]:
# note that the outputs of the Bayesian optimization runs are stored at the following link:

# https://drive.google.com/drive/folders/1WN8uEkxRbyV5DAnJuZWfvAN0BRyJkbLl?usp=sharing

# once downloaded, we perform the three diagnostic experiments in this notebook



In [ ]:
def get_results_filename(model, dataset, path='path/to/data'):
    local_files = os.listdir(path)
    for file_ in local_files:
        if file_.startswith(f'{model}_{dataset}') and not file_.endswith('config.json'):
            return file_


In [ ]:
datasets = ['bioconcentration', 'weather', 'facebook', 'abalone']



model = 'deep_ensemble'
path = 'data'


title_size = 22
label_size = 22
tick_size = 20

fig = plt.figure(figsize=(20, 3.5))
outer = gridspec.GridSpec(1, 4, wspace=0.2, hspace=0.4)

# per dataset
for i in range(4):

    dataset = datasets[i]
    print(dataset)
    filename = get_results_filename(model, dataset, path)
    with open(f'{path}/{filename}') as f:
        results = json.load(f)

    betas = []
    eval_losses = []
    div_losses = []
    w_ind_losses = []
    for run in results.keys():
        eval_losses.append(results[run]['ens_loss'])
        div_losses.append(results[run]['div_loss'])
        w_ind_losses.append(results[run]['w_ind_loss'])
    inner = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec=outer[i],
                                             wspace=0.15, hspace=0.25)

    # loss vs diversity
    for j in range(2):

        # plot loss
        ax = plt.Subplot(fig, inner[0])
        ax.set_title(dataset.capitalize(), fontsize=title_size)
        m = np.mean(eval_losses, axis=0)
        s = np.std(eval_losses, axis=0)
        ax.plot(results[run]['beta'], m, color=red, marker='o', ms=3)
        ax.fill_between(results[run]['beta'], m - s, y2=m + s, color=red,
                            alpha=0.25)
        ax.set_xticks([])
        upper = max(m + s)
        lower = min(m-s)
        eps = (upper - lower)/10
        y_ticks = [lower, (lower+upper)/2, upper]
        ax.set_ylim([lower - eps, upper + eps])
        ax.set_yticks(y_ticks)
        ax.set_yticklabels([round(tick, 3) for tick in y_ticks])
        if i == 0:
          ax.set_ylabel('MSE', fontsize=label_size)
          ax.get_yaxis().set_label_coords(-0.21,0.5)
        ax.tick_params(axis='both', which='major', labelsize=tick_size)
        fig.add_subplot(ax)


        # plot diversity
        ax = plt.Subplot(fig, inner[1])
        m = np.mean(div_losses, axis=0)
        s = np.std(div_losses, axis=0)
        ax.plot(results[run]['beta'], m, color=green, marker='o', ms=3)
        ax.fill_between(results[run]['beta'], m - s, y2=m + s, color=green,
                            alpha=0.25)
        upper = max(m + 0.2*s)
        lower = max([min(m-s),0])
        eps = (upper - lower)/10
        y_ticks = [lower, (lower+upper)/2, upper]
        ax.set_ylim([lower - eps, upper + eps])
        ax.set_yticks(y_ticks)
        ax.set_yticklabels([round(tick, 3) for tick in y_ticks])
        ax.set_xticks([0.0, 0.2, 0.4, 0.6, 0.8, 1])
        if i == 0:
          ax.set_ylabel('Diversity', fontsize=label_size)
          ax.get_yaxis().set_label_coords(-0.21,0.5)
        ax.tick_params(axis='both', which='major', labelsize=tick_size)
        # if i > 1 and j ==1:
        ax.set_xlabel(r'$\beta$', fontsize=label_size+3)
        fig.add_subplot(ax)

plt.savefig(f'Diversity_loss_{model}_neurips.pdf', dpi=1200)
fig.show()

In [ ]:
# performs the dropout experiment and plots results

def shuffle_along_axis(a, axis):
    idx = np.random.rand(*a.shape).argsort(axis=axis)
    return np.take_along_axis(a,idx,axis=axis)

# run dropout experiment
props = [0.2, 0.4, 0.6]
n_iter = 100


title_size = 22
label_size = 22
tick_size = 20
lwd = 3
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(17,3))
for d, dataset in enumerate(datasets):
    print(dataset)
    filename = get_results_filename(model, dataset, path)
    with open(f'{path}/{filename}') as f:
        results = json.load(f)
    ens_loss_drop_full_ls = [[] for _ in props]

    ens_loss_b_full_ls = []

    mse_full_ls = []
    for run in results.keys():
      # ens_loss_drop_ls = []
      ens_loss_drop_ls = [[] for _ in props]
      ens_loss_b_ls = []

      mse_ls = []
      labels = np.array(results[run]['targets'])[0]
      ind_preds = np.array(results[run]['ind_preds']) # shape = (num_beta_vals, test_set_size, ensemble_size)
      for idx, beta in enumerate(results[run]['beta']):
        # get decomposition of biased preds for this value of beta
        biased_preds_ind = ind_preds[idx]
        biased_preds_ens = biased_preds_ind.mean(1)
        ens_losses_b = (biased_preds_ens - labels)**2
        w_ind_losses_b = np.mean((biased_preds_ind[:,:,0] - labels[:,None])**2, axis=1)
        divs_b = w_ind_losses_b - ens_losses_b
        ens_loss_b_ls.append(np.mean(ens_losses_b))


        # get mse of raw preds
        mse = np.mean((ind_preds[idx].mean(1) - labels)**2)
        mse_ls.append(mse)

        # get dropout preds
        # for each drop proportion being considered
        for c, drop_prop in enumerate(props):
            num_drop = int(biased_preds_ind.shape[1] * drop_prop)
            num_keep = biased_preds_ind.shape[1] - num_drop
            drop_prop_losses = []
            # run n_iter iterations
            for i in range(n_iter):
                mask = np.ones(biased_preds_ind.shape)
                mask[:,:num_drop] = 0
                mask = shuffle_along_axis(mask, 1)
                dropped_ind_preds = mask * biased_preds_ind
                dropped_ens_pred = dropped_ind_preds.sum(1)/num_keep
                ens_losses_drop = (dropped_ens_pred - labels)**2
                drop_prop_losses.append(np.mean(ens_losses_drop)) # avg test set loss for this iter
            ens_loss_drop_ls[c].append(np.mean(drop_prop_losses)) # mean avg test set loss across all iter


      # store results for this run
      ens_loss_b_full_ls.append(ens_loss_b_ls)
      for c in range(len(props)):
        ens_loss_drop_full_ls[c].append(ens_loss_drop_ls[c])

      mse_full_ls.append(mse_ls)

    # plot results for this dataset
    curr_ax = ax[d]
    curr_ax.set_title(dataset.capitalize(), fontsize=title_size)
    for c, prop in enumerate(props):
        # m_drop = np.mean(ens_loss_drop_full_ls[c], axis=0)
        change = np.array(ens_loss_drop_full_ls[c]) - np.array(ens_loss_b_full_ls)
        m_drop = np.mean(change, axis=0)
        max_ = max(props)
        min_ = min(props)
        upper = max_ - min_
        col = plt.cm.Greens((prop-min_)*((1-upper)/upper) + upper)
        min_col = plt.cm.Greens(((max_ - min_)/2)*((1-upper)/upper) + upper)
        s_drop = np.std(change, axis=0)
        curr_ax.plot(results[run]['beta'], m_drop, color=col, label=f'With {prop} dropout', linewidth=lwd)
        curr_ax.fill_between(results[run]['beta'], m_drop - s_drop, y2=m_drop + s_drop, color=min_col,
                            alpha=0.2)
        upper = max(m_drop + s_drop)
        lower = 0
        eps = (upper - lower)/10
        y_ticks = [lower, (lower+upper)/2, upper]
        curr_ax.set_ylim([lower - eps, upper + eps])
        curr_ax.set_yticks(y_ticks)
        curr_ax.set_yticklabels([round(tick, 2) for tick in y_ticks])

    curr_ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])

    if d == 0:
      curr_ax.set_ylabel('Increase in\ntest error', fontsize=label_size)
      curr_ax.get_yaxis().set_label_coords(-0.15,0.5)
    curr_ax.tick_params(axis='both', which='major', labelsize=tick_size)

    curr_ax.set_xlabel(r'$\beta$', fontsize=label_size+3)

    if d == 0:
      leg = curr_ax.legend(loc=(0.02,0.3), prop={'size': 18})
      for line in leg.get_lines():
          line.set_linewidth(4)


fig.tight_layout()
plt.savefig(f'Dropout_{model}_neurips.pdf', dpi=1200)
plt.show()

In [ ]:
# performs

title_size = 22
label_size = 22
tick_size = 20
lwd = 3
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(17, 3))
for d, dataset in enumerate(datasets):
    filename = get_results_filename(model, dataset, path)
    with open(f'{path}/{filename}') as f:
        results = json.load(f)
    ens_loss_db_full_ls = []
    ind_loss_db_full_ls = []
    div_db_full_ls = []

    ens_loss_b_full_ls = []
    ind_loss_b_full_ls = []
    div_b_full_ls = []

    mse_full_ls = []
    for run in results.keys():
      ens_loss_db_ls = []
      ind_loss_db_ls = []
      div_db_ls = []

      ens_loss_b_ls = []
      ind_loss_b_ls = []
      div_b_ls = []

      mse_ls = []
      labels = np.array(results[run]['targets'])[0]
      ind_preds = np.array(results[run]['ind_preds'])[:,:,:,0] # shape = (num_beta_vals, test_set_size, ensemble_size)
      for idx, beta in enumerate(results[run]['beta']):
        # get decomposition of debiased preds for this value of beta
        ind_bias = ind_preds[idx].mean(0) - ind_preds[idx].mean(0).mean()
        debiased_preds_ind = ind_preds[idx] - ind_bias
        debiased_preds_ens = debiased_preds_ind.mean(1)
        ens_losses_db = (debiased_preds_ens - labels)**2
        w_ind_losses_db = np.mean((debiased_preds_ind - labels[:,None])**2, axis=1)
        divs_db = w_ind_losses_db - ens_losses_db
        ens_loss_db_ls.append(np.mean(ens_losses_db))
        ind_loss_db_ls.append(np.mean(w_ind_losses_db))
        div_db_ls.append(np.mean(divs_db))

        # get decomposition of biased preds for this value of beta
        biased_preds_ind = ind_preds[idx]
        biased_preds_ens = biased_preds_ind.mean(1)
        ens_losses_b = (biased_preds_ens - labels)**2
        w_ind_losses_b = np.mean((biased_preds_ind - labels[:,None])**2, axis=1)
        divs_b = w_ind_losses_b - ens_losses_b
        ens_loss_b_ls.append(np.mean(ens_losses_b))
        ind_loss_b_ls.append(np.mean(w_ind_losses_b))
        div_b_ls.append(np.mean(divs_b))

        # get mse of raw preds
        mse = np.mean((ind_preds[idx].mean(1) - labels)**2)
        mse_ls.append(mse)

      # store results for this run
      ens_loss_db_full_ls.append(ens_loss_db_ls)
      ind_loss_db_full_ls.append(ind_loss_db_ls)
      div_db_full_ls.append(div_db_ls)

      ens_loss_b_full_ls.append(ens_loss_b_ls)
      ind_loss_b_full_ls.append(ind_loss_b_ls)
      div_b_full_ls.append(div_b_ls)

      mse_full_ls.append(mse_ls)

    # plot results for this dataset
    curr_ax = ax[d]
    curr_ax.set_title(dataset.capitalize(), fontsize=title_size)

    percent_decrease = 1 - np.array(div_db_full_ls)/np.array(div_b_full_ls)
    m = np.mean(percent_decrease, axis=0)
    s = np.std(percent_decrease, axis=0)
    curr_ax.plot(results[run]['beta'], m, color=green, label='Biased', linewidth=lwd)
    curr_ax.fill_between(results[run]['beta'], m - s, y2=m + s, color=green,
                        alpha=0.25)

    max_val = max(m + s)
    eps = max_val/10
    y_vals = [0, max_val/2, max_val]
    y_vals_viz = np.round(np.array(y_vals)*100).astype(int)
    curr_ax.set_ylim([0 - eps, max_val + eps])
    curr_ax.set_yticks(y_vals)
    curr_ax.set_yticklabels(y_vals_viz)
    curr_ax.set_xticks([0, 0.5, 1])

    if (d == 0):
      curr_ax.set_ylabel('\% Diversity\nDecrease', fontsize=label_size)
      curr_ax.get_yaxis().set_label_coords(-0.09,0.5)
    curr_ax.tick_params(axis='both', which='major', labelsize=tick_size)

    curr_ax.set_xlabel(r'$\beta$', fontsize=label_size+3)


fig.tight_layout()
plt.savefig(f'Debiased_{model}2.pdf', dpi=1200)
plt.show()
